In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, year, month, row_number, desc, count, sum as spark_sum, 
    when, coalesce
)
from pyspark.sql.window import Window
from pyspark.sql.types import StringType
from notebookutils import mssparkutils
import json

StatementMeta(, 4db8c251-2b02-4aa1-bebe-7ef84c347917, 3, Finished, Available, Finished)

In [2]:
# ============================================
# Configuración y carga de claves de negocio
# ============================================

LAKEHOUSE_BRONZE = "lh_bronze"
LAKEHOUSE_SILVER = "lh_silver"
NULL_THRESHOLD = 0.95

# Cargar configuración con claves de negocio
def load_business_keys_config():
    """Carga claves de negocio desde el archivo de configuración"""
    try:
        config_path = "Files/config/redata_config.json"
        content = mssparkutils.fs.head(config_path, 100000)
        config = json.loads(content)
        print("✅ Configuración cargada desde redata_config.json")
        return config.get("business_keys", {}), config.get("non_business_columns", {})
    except Exception as e:
        print(f"⚠️ Error cargando config: {e}")
        print("📋 Usando configuración por defecto")
        return get_default_business_keys(), get_default_non_business_columns()


def get_default_business_keys():
    """Configuración por defecto de claves de negocio"""
    return {
        "balance_balance_electrico": {
            "keys": ["geo_id", "datetime", "series_type", "metric_type"],
            "description": "Balance eléctrico con métricas anidadas"
        },
        "demanda_evolucion": {
            "keys": ["geo_id", "datetime", "series_type"],
            "description": "Evolución de la demanda"
        },
        "generacion_estructura_generacion": {
            "keys": ["geo_id", "datetime", "series_type"],
            "description": "Estructura de generación"
        },
        "generacion_estructura_generacion_emisiones_asociadas": {
            "keys": ["geo_id", "datetime", "series_type"],
            "description": "Generación con emisiones"
        },
        "generacion_estructura_renovables": {
            "keys": ["geo_id", "datetime", "series_type"],
            "description": "Desglose renovables"
        },
        "generacion_evolucion_renovable_no_renovable": {
            "keys": ["geo_id", "datetime", "series_type"],
            "description": "Renovable vs no renovable"
        }
    }


def get_default_non_business_columns():
    """Columnas que NO son claves de negocio"""
    return {
        "metadata_temporal": [
            "ingestion_timestamp",
            "series_last_update",
            "metric_last_update"
        ],
        "particiones": ["year", "month"],
        "decorativas": [
            "metric_icon",
            "metric_color",
            "series_icon",
            "series_color"
        ]
    }


# Cargar configuración al inicio
BUSINESS_KEYS_CONFIG, NON_BUSINESS_COLUMNS = load_business_keys_config()

# Inicializar Spark
spark = SparkSession.builder.appName("Bronze_to_Silver_v2").getOrCreate()

print("🚀 Transformación Bronze → Silver v2.0")
print(f"📂 Origen: {LAKEHOUSE_BRONZE}")
print(f"📂 Destino: {LAKEHOUSE_SILVER}")
print(f"🔑 Claves de negocio cargadas: {len(BUSINESS_KEYS_CONFIG)} tablas")
print(f"✅ SOLO TABLAS MENSUALES\n")


StatementMeta(, 4db8c251-2b02-4aa1-bebe-7ef84c347917, 4, Finished, Available, Finished)

✅ Configuración cargada desde redata_config.json
🚀 Transformación Bronze → Silver v2.0
📂 Origen: lh_bronze
📂 Destino: lh_silver
🔑 Claves de negocio cargadas: 6 tablas
✅ SOLO TABLAS MENSUALES



In [3]:
# ============================================
# Funciones para claves de negocio
# ============================================

def get_business_keys_for_table(bronze_table: str) -> tuple:
    """
    Extrae las claves de negocio para una tabla Bronze.
    
    Returns:
        tuple: (business_keys: list, description: str)
    """
    # Extraer category/widget del nombre de tabla
    # Ejemplo: "brz_redata_demanda_evolucion_month" → "demanda_evolucion"
    table_key = bronze_table.replace("brz_redata_", "").replace("_month", "")
    
    config = BUSINESS_KEYS_CONFIG.get(table_key)
    
    if config is None:
        print(f"⚠️ No se encontró configuración para {table_key}, usando claves genéricas")
        return ["geo_id", "datetime", "series_type"], "Configuración genérica"
    
    return config["keys"], config.get("description", "")


def get_all_ignored_columns() -> list:
    """Devuelve todas las columnas a ignorar en deduplicación"""
    ignored = []
    for category_cols in NON_BUSINESS_COLUMNS.values():
        ignored.extend(category_cols)
    return list(set(ignored))


def validate_business_keys(df, business_keys: list, table_name: str) -> dict:
    """
    Valida que las claves existan en el DataFrame.
    
    Returns:
        dict: Resultado de validación con estadísticas
    """
    missing_keys = [k for k in business_keys if k not in df.columns]
    
    if missing_keys:
        return {
            "valid": False,
            "error": f"Claves faltantes: {missing_keys}",
            "available_columns": df.columns
        }
    
    # Contar duplicados con estas claves
    duplicates = df.groupBy(business_keys).agg(count("*").alias("dup_count"))
    dup_groups = duplicates.filter(col("dup_count") > 1).count()
    
    if dup_groups > 0:
        # Calcular cuántos registros se eliminarán
        total_dup_records = duplicates.filter(col("dup_count") > 1) \
                                      .agg(spark_sum("dup_count")).first()[0]
        to_remove = total_dup_records - dup_groups
    else:
        to_remove = 0
    
    return {
        "valid": True,
        "business_keys": business_keys,
        "duplicate_groups": dup_groups,
        "records_to_remove": to_remove,
        "total_records": df.count()
    }

StatementMeta(, 4db8c251-2b02-4aa1-bebe-7ef84c347917, 5, Finished, Available, Finished)

In [4]:
# ============================================
# Funciones auxiliares
# ============================================

def get_bronze_tables():
    """Lista SOLO tablas Bronze MENSUALES (_month)"""
    all_tables = spark.sql(f"SHOW TABLES IN {LAKEHOUSE_BRONZE}").collect()
    return sorted([row.tableName for row in all_tables 
                   if row.tableName.startswith("brz_redata_") 
                   and row.tableName.endswith("_month")])


def calculate_null_percentage(df):
    """Calcula % nulos por columna"""
    total = df.count()
    if total == 0:
        return {}
    null_counts = df.select([
        spark_sum(when(col(c).isNull(), 1).otherwise(0)).alias(c) 
        for c in df.columns
    ]).collect()[0].asDict()
    return {c: (n / total) for c, n in null_counts.items()}


def get_columns_to_drop(df):
    """Identifica columnas a eliminar"""
    to_drop = []
    
    # 1. Decorativas
    decorative = NON_BUSINESS_COLUMNS.get("decorativas", [])
    to_drop.extend([c for c in decorative if c in df.columns])
    
    # 2. >95% nulos
    null_pct = calculate_null_percentage(df)
    to_drop.extend([c for c, p in null_pct.items() 
                    if p > NULL_THRESHOLD and c not in ['ccaa_name']])
    
    # 3. Redundantes (conservar _type)
    redundant = [
        ('series_type', 'series_title', 'series_id'),
        ('metric_type', 'metric_title', 'metric_id')
    ]
    
    for keep, *check in redundant:
        if keep in df.columns:
            sample = df.select([keep] + [c for c in check if c in df.columns]).limit(100)
            for col_check in check:
                if col_check in df.columns:
                    distinct = sample.select(keep, col_check).distinct().count()
                    total = sample.select(keep).distinct().count()
                    if distinct <= total * 1.1 and col_check not in to_drop:
                        to_drop.append(col_check)
    
    return list(set(to_drop))


def deduplicate_by_business_keys(df, business_keys: list):
    """
    ✅ NUEVA FUNCIÓN: Deduplica usando SOLO claves de negocio.
    Mantiene el registro con ingestion_timestamp más reciente.
    
    Esta es la clave para resolver el problema de duplicados.
    """
    if "ingestion_timestamp" not in df.columns:
        print("  ⚠️ No hay ingestion_timestamp, usando deduplicación simple")
        return df.dropDuplicates(business_keys)
    
    window = Window.partitionBy(business_keys).orderBy(desc("ingestion_timestamp"))
    
    df_deduped = df.withColumn("_row_num", row_number().over(window)) \
                   .filter(col("_row_num") == 1) \
                   .drop("_row_num")
    
    return df_deduped


def normalize_ids(df):
    """Normaliza IDs: NULL si son placeholders"""
    if 'metric_id' in df.columns and 'metric_group_id' in df.columns:
        df = df.withColumn("metric_id",
            when(col("metric_id") == col("metric_group_id"), None)
            .otherwise(col("metric_id")))
    
    if 'series_id' in df.columns and 'series_group_id' in df.columns:
        df = df.withColumn("series_id",
            when(col("series_id") == col("series_group_id"), None)
            .otherwise(col("series_id")))
    
    return df


def add_partition_columns(df):
    """Agrega year/month desde datetime"""
    return df.withColumn("year", year(col("datetime"))) \
             .withColumn("month", month(col("datetime")))


def should_partition(table_name, record_count):
    """Decide estrategia de particionamiento"""
    if '_month' in table_name:
        return record_count > 500, ["year"]
    return False, []

StatementMeta(, 4db8c251-2b02-4aa1-bebe-7ef84c347917, 6, Finished, Available, Finished)

In [5]:
# ============================================
# Transformación de metadata
# ============================================

def copy_metadata_table(bronze_table):
    """Copia metadata sin transformaciones"""
    print(f"\n{'='*80}")
    print(f"📋 {bronze_table} (METADATA)")
    print(f"{'='*80}")
    
    df = spark.table(f"{LAKEHOUSE_BRONZE}.{bronze_table}")
    original = df.count()
    print(f"  📊 Originales: {original:,}")
    
    # Deduplicar por todas las columnas excepto timestamp
    key_cols = [c for c in df.columns if c != 'metadata_extraction_timestamp']
    if key_cols:
        window = Window.partitionBy(key_cols).orderBy(desc("metadata_extraction_timestamp"))
        df_clean = df.withColumn("row_num", row_number().over(window)) \
                     .filter(col("row_num") == 1) \
                     .drop("row_num")
        final = df_clean.count()
        print(f"  🔄 Deduplicados: {final:,} (-{original - final:,})")
    else:
        df_clean = df
        final = original
    
    # Escribir
    silver_table = bronze_table.replace("brz_", "slv_")
    df_clean.write.format("delta").mode("overwrite") \
           .option("overwriteSchema", "true") \
           .saveAsTable(f"{LAKEHOUSE_SILVER}.{silver_table}")
    
    print(f"  ✅ Copiada a: {silver_table}")
    
    return {
        "bronze_table": bronze_table,
        "silver_table": silver_table,
        "original_records": original,
        "final_records": final,
        "duplicates_removed": original - final,
        "columns_dropped": 0,
        "final_columns": len(df_clean.columns),
        "partitioned": False,
        "partition_cols": [],
        "verification_passed": True
    }

StatementMeta(, 4db8c251-2b02-4aa1-bebe-7ef84c347917, 7, Finished, Available, Finished)

In [6]:
# ============================================
# Transformación principal de tablas
# ============================================

def transform_table(bronze_table):
    """
    ✅ VERSIÓN MEJORADA: Transforma tabla con claves de negocio explícitas
    """
    print(f"\n{'='*80}")
    print(f"📋 {bronze_table}")
    print(f"{'='*80}")
    
    # 1. Obtener claves de negocio para esta tabla
    business_keys, description = get_business_keys_for_table(bronze_table)
    print(f"  📖 {description}")
    print(f"  🔑 Claves de negocio: {', '.join(business_keys)}")
    
    # 2. Leer desde Bronze
    df = spark.table(f"{LAKEHOUSE_BRONZE}.{bronze_table}")
    original = df.count()
    print(f"  📊 Registros originales: {original:,}")
    
    # 3. Validar claves de negocio
    validation = validate_business_keys(df, business_keys, bronze_table)
    
    if not validation["valid"]:
        print(f"  ❌ Error: {validation['error']}")
        print(f"  📋 Columnas disponibles: {validation['available_columns']}")
        return None
    
    if validation["duplicate_groups"] > 0:
        print(f"  🔍 Grupos duplicados detectados: {validation['duplicate_groups']:,}")
        print(f"  🗑️ Registros a eliminar: {validation['records_to_remove']:,}")
    else:
        print(f"  ✅ Sin duplicados detectados")
    
    # 4. ✅ DEDUPLICAR POR CLAVES DE NEGOCIO
    df = deduplicate_by_business_keys(df, business_keys)
    after_dedup = df.count()
    removed = original - after_dedup
    
    if removed > 0:
        print(f"  🔄 Después de deduplicar: {after_dedup:,} (-{removed:,})")
    
    # 5. Normalizar IDs
    df = normalize_ids(df)
    print(f"  🔢 IDs normalizados")
    
    # 6. Eliminar columnas
    cols_drop = get_columns_to_drop(df)
    if cols_drop:
        print(f"  🗑️ Eliminar ({len(cols_drop)}): {', '.join(cols_drop)}")
        df = df.drop(*cols_drop)
    print(f"  ✅ Columnas finales: {len(df.columns)}")
    
    # 7. Particiones
    df = add_partition_columns(df)
    
    # 8. Validar
    nulls_dt = df.filter(col("datetime").isNull()).count()
    nulls_val = df.filter(col("value").isNull()).count()
    if nulls_dt > 0 or nulls_val > 0:
        print(f"  ⚠️ {nulls_dt} datetime nulos, {nulls_val} value nulos")
    
    # 9. Escribir
    silver_table = bronze_table.replace("brz_", "slv_")
    should_part, part_cols = should_partition(bronze_table, after_dedup)
    
    writer = df.write.format("delta").mode("overwrite")
    if should_part:
        writer = writer.partitionBy(*part_cols)
        print(f"  📁 Particionado: {', '.join(part_cols)}")
    else:
        print(f"  📁 Sin particiones")
    
    writer.option("overwriteSchema", "true") \
          .saveAsTable(f"{LAKEHOUSE_SILVER}.{silver_table}")
    
    print(f"  ✅ Escrito: {silver_table}")
    
    # 10. Verificar que no quedan duplicados
    df_verify = spark.table(f"{LAKEHOUSE_SILVER}.{silver_table}")
    verify_validation = validate_business_keys(df_verify, business_keys, silver_table)
    
    if verify_validation["duplicate_groups"] > 0:
        print(f"  ⚠️ ADVERTENCIA: Aún hay {verify_validation['duplicate_groups']} duplicados")
        verification_passed = False
    else:
        print(f"  ✅ Verificación: Sin duplicados en Silver")
        verification_passed = True
    
    return {
        "bronze_table": bronze_table,
        "silver_table": silver_table,
        "business_keys": business_keys,
        "original_records": original,
        "final_records": after_dedup,
        "duplicates_removed": removed,
        "columns_dropped": len(cols_drop) if cols_drop else 0,
        "final_columns": len(df.columns) - 2,
        "partitioned": should_part,
        "partition_cols": part_cols if should_part else [],
        "verification_passed": verification_passed
    }

StatementMeta(, 4db8c251-2b02-4aa1-bebe-7ef84c347917, 8, Finished, Available, Finished)

In [7]:
# ============================================
# Main
# ============================================
def main():
    """Ejecuta transformación completa"""
    tables = get_bronze_tables()
    
    print(f"📦 Tablas Bronze (SOLO MENSUALES): {len(tables)}\n")
    
    if not tables:
        print("❌ No se encontraron tablas Bronze mensuales")
        return [], []
    
    results = []
    errors = []
    
    for table in tables:
        try:
            if table == "brz_redata_metadata":
                result = copy_metadata_table(table)
            else:
                result = transform_table(table)
            
            if result:
                results.append(result)
        except Exception as e:
            print(f"\n❌ Error en {table}: {str(e)}")
            import traceback
            traceback.print_exc()
            errors.append({"table": table, "error": str(e)})
    
    # Resumen
    print(f"\n{'='*80}")
    print("📊 RESUMEN")
    print(f"{'='*80}")
    
    total_orig = sum(r["original_records"] for r in results)
    total_final = sum(r["final_records"] for r in results)
    total_dup = sum(r["duplicates_removed"] for r in results)
    
    print(f"\n✅ Procesadas: {len(results)}")
    print(f"❌ Errores: {len(errors)}")
    
    if total_orig > 0:
        print(f"\n📈 Estadísticas:")
        print(f"  • Originales: {total_orig:,}")
        print(f"  • Finales: {total_final:,}")
        print(f"  • Duplicados eliminados: {total_dup:,}")
        print(f"  • Reducción: {(total_dup/total_orig*100):.1f}%")
    
    # Verificación de calidad
    failed_verification = [r for r in results if not r.get("verification_passed", True)]
    if failed_verification:
        print(f"\n⚠️ TABLAS CON DUPLICADOS REMANENTES ({len(failed_verification)}):")
        for r in failed_verification:
            print(f"  • {r['silver_table']}")
    else:
        print(f"\n✅ TODAS LAS TABLAS PASARON LA VERIFICACIÓN")
    
    partitioned = [r for r in results if r["partitioned"]]
    if partitioned:
        print(f"\n📁 Particionadas ({len(partitioned)}):")
        for r in partitioned:
            print(f"  • {r['silver_table']}: {', '.join(r['partition_cols'])}")
    
    if errors:
        print(f"\n❌ Errores:")
        for e in errors:
            print(f"  • {e['table']}: {e['error']}")
    
    print(f"\n{'='*80}")
    print("✅ Completado")
    print(f"{'='*80}\n")
    
    return results, errors


if __name__ == "__main__":
    results, errors = main()

StatementMeta(, 4db8c251-2b02-4aa1-bebe-7ef84c347917, 9, Finished, Available, Finished)

📦 Tablas Bronze (SOLO MENSUALES): 6


📋 brz_redata_balance_balance_electrico_month
  📖 Balance eléctrico con métricas anidadas
  🔑 Claves de negocio: geo_id, datetime, series_type, metric_type
  📊 Registros originales: 8,460
  🔍 Grupos duplicados detectados: 258
  🗑️ Registros a eliminar: 258
  🔄 Después de deduplicar: 8,202 (-258)
  🔢 IDs normalizados
  🗑️ Eliminar (8): metric_color, metric_icon, series_title, series_magnitude, metric_title, metric_id, metric_magnitude, series_id
  ✅ Columnas finales: 17
  📁 Particionado: year
  ✅ Escrito: slv_redata_balance_balance_electrico_month
  ✅ Verificación: Sin duplicados en Silver

📋 brz_redata_demanda_evolucion_month
  📖 Evolución de la demanda eléctrica
  🔑 Claves de negocio: geo_id, datetime, series_type
  📊 Registros originales: 640
  🔍 Grupos duplicados detectados: 20
  🗑️ Registros a eliminar: 20
  🔄 Después de deduplicar: 620 (-20)
  🔢 IDs normalizados
  🗑️ Eliminar (8): series_description, series_title, series_group_id, series_magnit